# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d100/6f49baaa5d38b70dfaa70c85c804ebc90370a013ae9e4fd2346b2115.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **St Petersburg, St.-Petersburg, Russia**, and the stations the data comes from are shown on the map below.

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(100,'6f49baaa5d38b70dfaa70c85c804ebc90370a013ae9e4fd2346b2115')

FileNotFoundError: File b'data/C2A2_data/BinSize_d100.csv' does not exist

In [ ]:
df2 = pd.read_csv('data/C2A2_data/BinnedCsvs_d100/6f49baaa5d38b70dfaa70c85c804ebc90370a013ae9e4fd2346b2115.csv')
df2.drop(['ID'],axis = 1,inplace = True)
df2['Year'] = df2['Date'].str.slice(start=0, stop=4, step = 1)  
df2['Date'] = [e[5:]for e in df2['Date'].values]
print(df2)



In [ ]:
def df2005_2014():    
    df2005_2014 = df2[df2.Year != '2015']
    df2005_2014.drop(['Element'],axis = 1,inplace = True)
    df2005_2014min = df2005_2014.groupby(['Date']).min()
    df2005_2014max = df2005_2014.groupby(['Date']).max()
    df2005_2014min = df2005_2014min.rename(columns={"Data_Value": "Min_T"})
    df2005_2014max = df2005_2014max.rename(columns={"Data_Value": "Max_T"})
    df2005_2014 = pd.concat([df2005_2014max,df2005_2014min],axis = 1)
    df2005_2014.drop(['Year'],axis = 1,inplace = True)
    df2005_2014 = df2005_2014[df2005_2014.index != '02-29']
    df2005_2014.rename(columns={'Min_T': 'Min_T_during_2005-2014', 'Max_T': 'Max_T_during_2005-2014'}, inplace = True)
    
    return df2005_2014

df2005_2014().head(3)

In [ ]:
def df2015():
    df2015 = df2[df2.Year == '2015']
    df2015min = df2015.groupby(['Date']).min()
    df2015max = df2015.groupby(['Date']).max()
    df2015min = df2015min.rename(columns={"Data_Value": "Min_T"})
    df2015max = df2015max.rename(columns={"Data_Value": "Max_T"})
    df2015 = pd.concat([df2015max,df2015min],axis = 1)
    df2015.drop(['Element','Year'],axis = 1,inplace = True)
    df2015.rename(columns={'Min_T': 'Min_T_2015', 'Max_T': 'Max_T_2015'}, inplace = True)
    return df2015

df2015().head(3)

In [ ]:
import numpy as np
def dfdelta():
    dfdelta = df2015()#pd.DataFrame({'Min_T' : [],'Max_T' : []})
    dfdelta['Min_T_2015'] = np.where((df2005_2014()['Min_T_during_2005-2014'] > df2015()['Min_T_2015'])
                     , df2015()['Min_T_2015'], np.nan)
    dfdelta['Max_T_2015'] = np.where((df2005_2014()['Max_T_during_2005-2014'] < df2015()['Max_T_2015'])
                     , df2015()['Max_T_2015'], np.nan)
    dfdelta.dropna(axis=0, thresh=1, subset=None, inplace=True)
    dfdelta.rename(columns={'Min_T_2015': 'Min_T_2015_breaking_point', 'Max_T_2015': 'Max_T_2015_breaking_point'}, inplace = True)         
    return dfdelta

dfdelta().head(3)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt

#https://matplotlib.org/xkcd/examples/api/fahrenheit_celsius_scales.html

def linegraph():
       
    observation_dates = np.arange('2015-01-01', '2016-01-01', dtype='datetime64[D]')
    observation_delta_dates = dfdelta().index
    observation_delta_dates = ['2015-'+ str(e) for e in observation_delta_dates]
    observation_delta_dates = np.asarray(observation_delta_dates, dtype='datetime64[D]', order=None)

    plt.figure(figsize=(16,8))
    plt.plot(observation_dates,df2005_2014()['Max_T_during_2005-2014'], color='red', alpha=0.5)
    plt.plot(observation_dates,df2005_2014()['Min_T_during_2005-2014'], color='blue', alpha=0.5)
    plt.scatter(observation_delta_dates, dfdelta()['Max_T_2015_breaking_point'], marker='o', color='red')
    plt.scatter(observation_delta_dates, dfdelta()['Min_T_2015_breaking_point'], marker='o', color ='blue')
    plt.box(on=None) 
    
    ax = plt.gca()   
    temp = [str(tick/10)+str(' °C') for tick in ax.get_yticks()]   
    ax.set_yticklabels(temp)  
    ax.xaxis.set_tick_params(labelsize=12)
    ax.yaxis.set_tick_params(labelsize=12)  
    ax.grid(color='black', linestyle='-', linewidth=0.1)
    ax.axis([observation_dates[0],observation_dates[-1],-400,400])
    
    myFmt = mdates.DateFormatter('%B')
    plt.gca().xaxis.set_major_formatter(myFmt)  
    plt.setp(ax.xaxis.get_majorticklabels(), ha="left", rotation = 30)
    plt.gca().fill_between(observation_dates,df2005_2014()['Max_T_during_2005-2014'], df2005_2014()['Min_T_during_2005-2014'], 
                       facecolor='black', 
                       alpha=0.5)
    
    plt.legend(loc=8, title=None, frameon=False, fontsize=12)
    plt.title('2015 temperature breaking points', fontsize=14)
    
    #plt.legend(loc=(0.5, 0.15), title=None, frameon=False)
    return plt.show()
    
linegraph()
